In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('/projects/cpjump1/jump/load_data/ctrls_images.parquet')

In [ ]:
df.columns

In [15]:
images_cols = [
 'FileName_OrigAGP',
 'FileName_OrigDNA',
 'FileName_OrigER',
 'FileName_OrigMito', 
 'FileName_OrigRNA']

In [22]:
metadata = df.iloc[7][images_cols].to_dict()


In [ ]:
metadata

In [ ]:
for img_path in df.iloc[7][images_cols].values:
    print(img_path)

In [ ]:
pd.DataFrame(df.iloc[7][images_cols].values, columns=["zs"])

In [ ]:
pd.read_csv('/projects/synsight/repos/phenoseeker/tmp/metadata.csv')

In [30]:
import numpy as np

In [ ]:
np.load('/projects/synsight/repos/phenoseeker/tmp/extracted_features.npy')